<a href="https://colab.research.google.com/github/pmopedro/computational-chess/blob/fabio/notebooks/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import joblib

In [2]:
!pip install chess
import chess

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
test_folder = '/content/gdrive/MyDrive/IA006_Projeto/test/'
test_info = test_folder + 'games_positions.csv'

df_test = pd.read_csv(test_info)

#### Funções utilitárias para processamento do FEN

In [5]:
def boardstate(fen):
    board = chess.Board(fen[0])
    fstr = str(fen[0])

    if board.has_kingside_castling_rights(chess.WHITE) == True:
        WCKI = 1
    else:
        WCKI = 0
    if board.has_queenside_castling_rights(chess.WHITE) == True:
        WCQ = 1
    else:
        WCQ = 0
    if board.is_check() == True:
        WCH = 1
    else:
        WCH = 0

    if board.has_kingside_castling_rights(chess.BLACK) == True:
        BCKI = 1
    else:
        BCKI = 0
    if board.has_queenside_castling_rights(chess.BLACK) == True:
        BCQ = 1
    else:
        BCQ = 0
    if board.was_into_check() == True:
        BCH = 1
    else:
        BCH = 0

    #f = [M, WCKI, WCQ, WCH, BCKI, BCQ, BCH]
    fw = [WCKI, WCQ, WCH]
    fb = [BCKI, BCQ, BCH]

    bstr = str(board)
    bstr = bstr.replace("p", "\ -1")
    bstr = bstr.replace("n", "\ -3")
    bstr = bstr.replace("b", "\ -4")
    bstr = bstr.replace("r", "\ -5")
    bstr = bstr.replace("q", "\ -9")
    bstr = bstr.replace("k", "\ -100")
    bstr = bstr.replace("P", "\ 1")
    bstr = bstr.replace("N", "\ 3")
    bstr = bstr.replace("B", "\ 4")
    bstr = bstr.replace("R", "\ 5")
    bstr = bstr.replace("Q", "\ 9")
    bstr = bstr.replace("K", "\ 100")
    bstr = bstr.replace(".", "\ 0")
    bstr = bstr.replace("\ ", ",")
    bstr = bstr.replace("'", " ")
    bstr = bstr.replace("\n", "")
    bstr = bstr.replace(" ", "")
    bstr = bstr[1:]
    bstr = eval(bstr)
    bstr = list(bstr)
    if "w" not in fstr:
        for i in range(len(bstr)):
            bstr[i] = bstr[i] * -1
        bstr.reverse()
        fs = fb
        fb = fw
        fw = fs


    BITBOARD = fw + fb + bstr


    return BITBOARD

In [6]:
def fen_to_board(fen):
    board = []
    for row in fen.split('/'):
        brow = []
        for c in row:
            if c == ' ':
                break
            elif c in '12345678':
                brow.extend( ['--'] * int(c) )
            elif c == 'p':
                brow.append( 'bp' )
            elif c == 'P':
                brow.append( 'wp' )
            elif c > 'Z':
                brow.append( 'b'+c.upper() )
            else:
                brow.append( 'w'+c )

        board.append( brow )
    return board

In [7]:
df_test.iloc[0, 6]

'2bqkbnr/5ppp/2Pp4/2p1p3/4P3/r1PPB3/1P2BPPP/R2QK1NR w KQk - 0 12'

In [8]:
fen_to_board(df_test.iloc[0,6])

[['--', '--', 'bB', 'bQ', 'bK', 'bB', 'bN', 'bR'],
 ['--', '--', '--', '--', '--', 'bp', 'bp', 'bp'],
 ['--', '--', 'wp', 'bp', '--', '--', '--', '--'],
 ['--', '--', 'bp', '--', 'bp', '--', '--', '--'],
 ['--', '--', '--', '--', 'wp', '--', '--', '--'],
 ['bR', '--', 'wp', 'wp', 'wB', '--', '--', '--'],
 ['--', 'wp', '--', '--', 'wB', 'wp', 'wp', 'wp'],
 ['wR', '--', '--', 'wQ', 'wK', '--', 'wN', 'wR']]

In [9]:
df_test.dropna(inplace=True)

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18114 entries, 0 to 18113
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    18114 non-null  int64 
 1   id            18114 non-null  object
 2   turns         18114 non-null  int64 
 3   winner        18114 non-null  object
 4   white_rating  18114 non-null  int64 
 5   black_rating  18114 non-null  int64 
 6   position      18114 non-null  object
dtypes: int64(4), object(3)
memory usage: 990.7+ KB


### Conversão FEN para board numérica para as features de entrada



In [11]:
data_features = df_test[['position']]

data_features = data_features.apply(boardstate, axis=1)
data_features = data_features.apply(pd.Series)
print(data_features.head())

   0   1   2   3   4   5   6    7   8   9   ...  60  61  62  63  64  65   66  \
0   1   1   0   1   0   0   0    0  -4  -9  ...   1   1   5   0   0   9  100   
1   0   0   0   0   0   0   0 -100   0  -5  ...   1   1   5   0   0   9    0   
2   1   1   0   0   0   0  -5    0  -4   0  ...   1   1   5   0   0   9  100   
3   0   0   1   1   1   0  -5    0   0   0  ...   1   1   5   0   4   0    0   
4   1   1   0   0   0   0  -5    0  -4  -9  ...   1   0   5   0   0   0  100   

   67   68  69  
0   0    3   5  
1   5  100   0  
2   0    0   5  
3   5  100   0  
4   0    0   5  

[5 rows x 70 columns]


### Conversão do ganhador do jogo para score de resultado:
* Branco vencedor - score = 10000
* Preto vencedor - score = -10000
* Empates - score = 0

In [12]:
def label_winner(row):
   if row['winner'] == 'white':
      return 10000
   if row['winner'] == 'black':
      return -10000
   return 0

data_labels = df_test.apply(label_winner, axis=1)
print(data_labels.head())

0    10000
1    10000
2    10000
3    10000
4   -10000
dtype: int64


In [13]:
import numpy as np
input2_columns = [0, 1, 2, 3, 4, 5]


inputboard = data_features.drop(columns=data_features.iloc[:, input2_columns])
inputboard = np.array(inputboard)
inputmeta = data_features.iloc[:, input2_columns]
inputmeta = np.array(inputmeta)

In [14]:
print(inputboard)

[[   0    0   -4 ...    0    3    5]
 [   0 -100    0 ...    5  100    0]
 [  -5    0   -4 ...    0    0    5]
 ...
 [  -5    0   -4 ...    0    0    5]
 [  -5   -3    0 ...    0    3    5]
 [  -5    0   -9 ...    0    0    5]]


In [15]:
print(inputmeta)

[[1 1 0 1 0 0]
 [0 0 0 0 0 0]
 [1 1 0 0 0 0]
 ...
 [1 1 0 0 0 0]
 [0 0 0 1 1 0]
 [1 1 0 1 1 0]]


In [16]:
data_features = data_features.to_numpy()
data_labels = data_labels.to_numpy()

#### Carregamento do modelo e inferência

In [17]:
import tensorflow as tf
import os
import numpy as np

#model_file = '/content/gdrive/MyDrive/IA006_Projeto/models/model.pkl'
model_file = '/content/gdrive/MyDrive/IA006_Projeto/models/model1.pkl'

data_model = joblib.load(model_file)

In [18]:
predictions = data_model.predict([(inputboard), (inputmeta)])

567/567 [==============================] - 4s 3ms/step


#### Verificação da acurácia do método utilizado para estimar a vitória/derrota do jogador.

In [19]:
# Conversão de predições
pred_new = np.full_like(predictions, 0)
count = 0
for i in predictions.flat:
  if (i > 0):
    pred_new[count] = 1
  elif (i < 0):
    pred_new[count] = -1
  else:
    pred_new[count] = 0
  count = count+1

pred_new = pred_new.transpose()

In [20]:
print(pred_new)

[[ 1.  1.  1. ...  1. -1.  1.]]


In [21]:
# Conversão de datalabels
labels_new = np.full_like(data_labels, 0)
count = 0
for i in data_labels.flat:
  if (i > 0):
    labels_new[count] = 1
  elif (i < 0):
    labels_new[count] = -1
  else:
    labels_new[count] = 0
  count = count+1

In [22]:
print(labels_new)

[ 1  1  1 ...  1  1 -1]


In [23]:
# Contagem do número de elementos iguais (Medida de acurácia do método)

print(np.sum(pred_new == labels_new) / pred_new.size)

0.514022303190902
